In [ ]:
!pip install -q --upgrade keras-cv
!pip install -q --upgrade keras  # Upgrade to Keras 3.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # @param ["tensorflow", "jax", "torch"]

import json
import math
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import losses
from keras import ops
from keras import optimizers
from keras.optimizers import schedules
from keras import metrics

import keras_cv

# Import tensorflow for `tf.data` and its preprocessing functions
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir_path = "file path"

In [ ]:
BATCH_SIZE = 16
IMAGE_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
tfds.disable_progress_bar()
train_generator = tf.keras.utils.image_dataset_from_directory(
    directory=dir_path,  color_mode='rgb',label_mode='int',
    image_size=IMAGE_SIZE,batch_size=BATCH_SIZE, shuffle=True,subset='training',validation_split=0.2,
    seed=25)
validation_generator = tf.keras.utils.image_dataset_from_directory(
    directory=dir_path, color_mode='rgb',
    label_mode='int',image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,shuffle=False,subset='validation',validation_split=0.2,
    seed=25)

In [ ]:
class_names = train_generator.class_names
print(class_names)

In [ ]:
train_ds_shuff = train_generator.shuffle(
    3*BATCH_SIZE, reshuffle_each_iteration=True
)
images = next(iter(train_ds_shuff.take(1)))[0]
images.shape

In [ ]:
keras_cv.visualization.plot_image_gallery(images, value_range=(0, 255))

In [ ]:
keras.backend.clear_session()

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_generator.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds = validation_generator.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
print(keras_cv.models.ImageClassifier.presets.keys())

In [ ]:
model = keras_cv.models.ImageClassifier.from_preset(
    "resnet50_v2_imagenet", num_classes=len(class_names

In [ ]:
model = keras_cv.models.ImageClassifier.from_preset(
    "resnet50_v2_imagenet", num_classes=len(class_names

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, AdamW
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

checkpoint_filepath = 'model/weights-improvement-{epoch:02}-{val_loss:.4f}.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                     filepath=checkpoint_filepath,
                     verbose=2, monitor='val_acc', mode='max',
                     save_best_only=True
                 )
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=15,
                        verbose=2, restore_best_weights=True)
model.compile(optimizer=AdamW(learning_rate=0.001),loss='sparse_categorical_crossentropy',
              metrics = ['acc'])

In [ ]:
model.summary()

In [ ]:
r = model.fit(normalized_train_ds,
              validation_data=normalized_test_ds, callbacks=[model_checkpoint_callback],
              epochs=100)

In [ ]:
model.load_weights("model file path")


In [ ]:
logloss, acc = model.evaluate(normalized_test_ds)
print("Accuracy: ", acc)
print("Logloss: ", logloss)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['acc'], label='Accuracy')
plt.plot(r.history['val_acc'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
from PIL import Image
import numpy as np

# Load the image
path = "path of image"
image = Image.open(path)

# Convert the image to a numpy array
image = np.array(image)

# Normalize the image
image = image / 255.0

# Add a batch dimension
image = np.expand_dims(image, axis=0)

In [ ]:
resizing = keras_cv.layers.Resizing(
    IMAGE_SIZE[0], IMAGE_SIZE[1], crop_to_aspect_ratio=True
)

In [ ]:
np_im_rs = resizing(image)
np_im_rs.shape

In [ ]:
predictions = model.predict(np_im_rs)
predictions

In [ ]:
print("Top class is:", class_names[np.argmax(predictions[0])])


In [ ]:
model.save_weights('/content/drive/MyDrive/Internship/anim_class_model.weights.h5')